In [0]:
fileroot = 'clinicaltrial_2021'
 
import os
os.environ ['fileroot'] = fileroot

In [0]:
#Checking the file has been uploaded

dbutils.fs.ls("/FileStore/tables/")

Out[10]: [FileInfo(path='dbfs:/FileStore/tables/FaultDataset-1.csv', name='FaultDataset-1.csv', size=1703184, modificationTime=1678731499000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-2.csv', name='FaultDataset-2.csv', size=1703184, modificationTime=1678731542000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-3.csv', name='FaultDataset-3.csv', size=1703184, modificationTime=1678813438000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-4.csv', name='FaultDataset-4.csv', size=1703184, modificationTime=1678814094000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-5.csv', name='FaultDataset-5.csv', size=1703184, modificationTime=1678895444000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset-6.csv', name='FaultDataset-6.csv', size=1703184, modificationTime=1678895551000),
 FileInfo(path='dbfs:/FileStore/tables/FaultDataset.csv', name='FaultDataset.csv', size=1703184, modificationTime=1678726368000),
 FileInfo(path='dbfs:/FileStore/tables/Occupancy_Detectio

In [0]:
#Copy a file from Databricks file system to local file system using dbutils.fs.cp() function".

dbutils.fs.cp("/FileStore/tables/clinicaltrial_2021.zip", "file:/tmp/")

Out[11]: True

In [0]:
#Check the tmp directory to see if clinicaltrial_2021.zip is in it

In [0]:
%sh
ls /tmp/

Rserv
Rtmp7MEKKr
chauffeur-daemon-params
chauffeur-daemon.pid
chauffeur-env.sh
clinicaltrial_2021.zip
custom-spark.conf
driver-daemon-params
driver-daemon.pid
driver-env.sh
hsperfdata_root
pharma.zip
systemd-private-8a9b56d39423442c923ad14c209364e1-apache2.service-6jzoDf
systemd-private-8a9b56d39423442c923ad14c209364e1-ntp.service-BvKv8h
systemd-private-8a9b56d39423442c923ad14c209364e1-systemd-logind.service-XMRmpg
systemd-private-8a9b56d39423442c923ad14c209364e1-systemd-resolved.service-P2UKlh
tmp.VptDvKWzIS


In [0]:
%sh

unzip -d /tmp/ /tmp/clinicaltrial_2021.zip

Archive:  /tmp/clinicaltrial_2021.zip
  inflating: /tmp/clinicaltrial_2021.csv  


In [0]:
#Moving the unzip files to DBFS

dbutils.fs.mkdirs("FileStore/tables/clinicaltrial_2021")

Out[15]: True

In [0]:
#Move the content of the local file directory into the newly created DBFS
dbutils.fs.mv("file:/tmp/clinicaltrial_2021.csv", "/FileStore/tables/clinicaltrial_2021.csv", True)


Out[16]: True

In [0]:
dbutils.fs.ls('FileStore/tables/clinicaltrial_2021.csv/')

Out[17]: [FileInfo(path='dbfs:/FileStore/tables/clinicaltrial_2021.csv/clinicaltrial_2021.csv', name='clinicaltrial_2021.csv', size=50359696, modificationTime=1682528035000)]

In [0]:
dbutils.fs.cp("/FileStore/tables/pharma.zip", "file:/tmp/")

Out[18]: True

In [0]:

%sh

unzip -d /tmp/ /tmp/pharma.zip

Archive:  /tmp/pharma.zip
  inflating: /tmp/pharma.csv         


In [0]:
%sh

ls /tmp/pharma.csv

/tmp/pharma.csv


In [0]:
dbutils.fs.mkdirs("FileStore/tables/pharma.csv")

Out[21]: True

In [0]:
dbutils.fs.mv("file:/tmp/pharma.csv", "/FileStore/tables/pharma.csv", True)

Out[22]: True

In [0]:
dbutils.fs.ls("FileStore/tables/pharma.csv/")

Out[23]: [FileInfo(path='dbfs:/FileStore/tables/pharma.csv/pharma.csv', name='pharma.csv', size=678999, modificationTime=1682528037000)]

In [0]:
len(dbutils.fs.ls("FileStore/tables/pharma.csv/"))

Out[24]: 1

In [0]:
#installs the Python package "Bokeh" using the pip package installer.
%pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
# Setting clinicaltrialdata = 2021

clinicaldata = 2021

In [0]:
# Creating a Resilient Distributed Dataset (RDD) by loading a CSV file and storing it in a newly created variable initialized as empty.

clinicaltrial_2021 = sc.textFile("/FileStore/tables/clinicaltrial_"+str(clinicaldata)+".csv")

pharma = sc.textFile("/FileStore/tables/pharma.csv")

In [0]:
#Checking the Created RDD
clinicaltrial_2021.take(5)

Out[3]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|']

In [0]:
#NUMBER OF DISTINCT STUDIES EXPLICITLY IN THE DATASET

#One way to remove the first column of a dataset using iteration is to check each row and exclude the first element. Additionally, you can use an "if" statement to disregard any null values present in the dataset.

Remove_First_Column = clinicaltrial_2021.mapPartitionsWithIndex(lambda id_x, iter: list(iter)[1:] if(id_x == 0) else iter)

#To Check the number of the Distinct studies.  

Remove_First_Column.distinct().count()

Out[4]: 387261

In [0]:
#Use a lambda function, to extract the first coumn that was initially dropped.

split_data = Remove_First_Column.map(lambda line: line.split("|"))

#To check the column types
Selecting_types = split_data.map(lambda line: line[5])

#Arranging it from most frquent to least frequent by adding 1

Highest_Value = Selecting_types.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: -x[1])

#Collecting the values
Highest_Value.collect()

Out[5]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
#Getting the previous Dropped column and splitting it using lambda function
Splitting = Remove_First_Column.map(lambda line: line.split("|"))

#Getting only the Condition columns 
Selecting_Conditions = Splitting.map(lambda line: line[7])

#Splitting it with ,
Splitting_Conditions = Selecting_Conditions.flatMap(lambda line: line.split(","))

#Adding a value 1 to all the values and using reduce by key function getting the count finally sorting it by descending order 
Counting_Top_Values = Splitting_Conditions.map(lambda x: (x,1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1],False)

#Getting the First Row
First_Element = Counting_Top_Values.first()

#Ignoring the First Row 
Removing_First_element = Counting_Top_Values.filter(lambda x: x != First_Element)

#Taking only 5 elements
Removing_First_element.take(5)

Out[6]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
#Split the pharma dataset of the second column 
Pharma_Files = pharma.map(lambda y : y.split(',')[1]).map(lambda x: x.replace('"',''))

#Split the values in the 'clinicaltrial_2021' table into individual elements or columns for easier data processing and analysis.
Main_data =  clinicaltrial_2021.map(lambda s : s.split("|"))

#Retrieve only the values from the second column of the resulting split dataset from the previous step for further data processing.
Getting_Sponsor = Main_data.map(lambda t : t[1])

#Subtract the 'Pharma_Files' dataset from the 'clinicaltrial_2021' dataset and group the resulting dataset by key.
subtracting =  Getting_Sponsor.subtract(Pharma_Files).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1], False)

#Retrieve only the top 10 values of the resulting dataset for further analysis or output.
subtracting.take(10)

Out[7]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
#Extract only the 'Complete' and 'Status' columns from the 'clinicaltrial_2021' table.
Month_year = clinicaltrial_2021.map(lambda x: x.split("|")).map(lambda y : (y[2],y[4]))

#Retrieve only the rows that have a 'Completed' value in the 'Status' column of the dataset.
completed_list = Month_year.filter(lambda s :  'Completed' in s)

#Reduce the 'yearly_completed_trials' dataset by summing the values for each 'Complete' value
extracted  = completed_list.map(lambda op :op[1]).filter(lambda s :'2021' in s).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x+y)

#Split the first column of the dataset to extract individual elements. 
obligating = extracted.map(lambda x: (x[0].split(' ')[0], x[1]))

#Collecting the result 
obligating.collect()

Out[8]: [('Jan', 1131),
 ('Jun', 1094),
 ('Aug', 700),
 ('Apr', 967),
 ('Mar', 1227),
 ('May', 984),
 ('Feb', 934),
 ('Jul', 819),
 ('Oct', 187),
 ('Sep', 528)]